In [3]:
!pip install icecream

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import random
from icecream import ic
import numpy as np
from tqdm import tqdm


ModuleNotFoundError: No module named 'icecream'

In [ ]:
#suggetions to improve your code
def fitness_order_modified(solutions, data):
    """Sort solutions by tour length ascending (shorter is better)."""
    # fix: avoiding unnecessary recomputation of fitness
    scored = []
    for s in solutions:
        scored.append((s, route_length(s, data)))

    scored.sort(key=lambda x: x[1])
    return [s for s, _ in scored]

def deduplicate_population(population):
    """remove duplicates to obtain more diversity in the population"""
    return list({tuple(sol): sol for sol in population}.values())

def tournament_selection(population, data, k=3):
    """select the best individual among k random individuals from the population"""
    candidates = random.sample(population, k)
    best = min(candidates, key=lambda s: route_length(s, data))
    return best


In [ ]:
"""Create a random permutation of cities [0..n-1]."""
def random_solution(n):
    perm = list(range(n))
    random.shuffle(perm)
    return perm


"""Compute the total distance of the tour given by 'solution'."""
def route_length(solution, data):
    n = len(solution)
    total = 0.0
    #if a-b-c-d-a ,range(n); else if a-b-c-d , range(n-1)
    #b = solution[(i + 1) % n]; b = solution[i + 1]
    for i in range(n - 1):
        a = solution[i]
        b = solution[i + 1]
        total += float(data[a][b])
    return total

"""Sort solutions by tour length ascending (shorter is better)."""
def fitness_order(solutions, data):
    scored = [(s, route_length(s, data)) for s in solutions]
    scored.sort(key=lambda x: x[1])
    return [s for s, _ in scored]



"""Using Standard OX crossover to create a child solution from two parents."""
def ox_crossover(parent1, parent2):
    n = len(parent1)
    a, b = sorted(random.sample(range(n), 2))
    #avoid a==b
    if a == b:
        b = (a + 1) % n
    child = [None] * n
    child[a:b+1] = parent1[a:b+1]

    used = set(child[a:b+1])
    write = (b + 1) % n
    for g in parent2:
        if g not in used:
            child[write] = g
            write = (write + 1) % n

    return child

"""Perform swap mutation on perm solution"""
def swap(solution):
    n = len(solution)
    a, b = random.sample(range(n), 2)
    #be sure to have different indexes
    while a == b:
        a, b = random.sample(range(n), 2)
    sol = solution[:]
    sol[a], sol[b] = sol[b], sol[a]
    return sol

"""Fitness-proportionate selection (shorter length -> higher weight)."""
"""For example, [10, 12, 13], max_len = 13, scores = [3, 1, 0],so probability of selecting each is [3/4, 1/4, 0]."""


def roulette(population, data):
    lengths = [route_length(s, data) for s in population]
    mx, mn = max(lengths), min(lengths)

    if mx == mn:
        # All have the same length, select randomly
        return random.choice(population)

    scores = [mx - L for L in lengths]  # > 0
    #introduce a small constant to avoid zero probability
    scores = [mx - L + 1e-6 for L in lengths]

    total = sum(scores)
    r, acc = random.random() , 0.0
    #r is a value between 0 but acc sums unnormalized scores so we have to scale r or normalize scores
    # opt1: r = random.random() * total
    #opt2: probs = [s / total for s in scores]


    for s, w in zip(population, scores):
        acc += w
        if acc >= r:
            return s
    return population[-1]

In [ ]:
early_stopping_no_improve = 500

"""Run GA for a single problem with OX crossover and swap mutation."""
def solve(data, populationSize, maxSteps, mutationProbability=0.2, seed=None):
    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)
    n = data.shape[0]
    population = [random_solution(n) for _ in range(populationSize)]

    population = [random_solution(n) for _ in range(populationSize)]
    lengths = [route_length(s, data) for s in population]

    best_distance = min(lengths)
    best_solution = population[lengths.index(best_distance)][:]

    history = [best_distance]
    patience = 200        
    restart_frac = 0.30   
    no_improve = 0

    for _ in tqdm(range(maxSteps)):
        """
        # select elites to avoid to lose the best solution found
        lengths = [route_length(s, data) for s in population]
        elite_idx = np.argsort(lengths)[:2] 
        elite = [population[i][:] for i in elite_idx]
        new_population = []
        events = max(1, populationSize // 6)
        new_population = []
        for _ in range(populationSize - len(elite)):  # lascia spazio all’élite
            parent1 = roulette(population, data)
            parent2 = roulette(population, data)
            child = ox_crossover(parent1, parent2)
            if random.random() < mutationProbability:
                child = swap(child)
            new_population.append(child)
    
        population = elite + new_population
        curr_idx = np.argmin(lengths)
        best_distance = lengths[curr_idx]
        best_solution = population[curr_idx][:]
        """
        events = max(1, populationSize // 6)
        for _e in range(events):

            p1 = roulette(population, data)
            p2 = roulette(population, data)

            child = ox_crossover(p1, p2)
            if random.random() < mutationProbability:
                child = swap(child)
            Lc = route_length(child, data)

            worst_idx = max(range(populationSize), key=lambda i: lengths[i])
            if Lc < lengths[worst_idx]:
                population[worst_idx] = child
                lengths[worst_idx] = Lc
        # update best
        curr_idx = min(range(populationSize), key=lambda i: lengths[i])
        curr_best = lengths[curr_idx]
        if curr_best < best_distance:
            best_distance = curr_best
            best_solution = population[curr_idx][:]
            no_improve = 0
        else:
            no_improve += 1
            
        #we can add also an early stop condition like this after every iteration with early_stopping_no_improve=500:
        if no_improve >= early_stopping_no_improve: 
            break
        history.append(best_distance)

        # restart if no improvement and keep elites
        if no_improve >= patience:
            inject = max(1, int(populationSize * restart_frac))
            worst_indices = sorted(range(populationSize), key=lambda i: lengths[i], reverse=True)[:inject]
            for idx in worst_indices:
                population[idx] = random_solution(n)
                lengths[idx] = route_length(population[idx], data)
            curr_idx = min(range(populationSize), key=lambda i: lengths[i])
            best_distance = lengths[curr_idx]
            best_solution = population[curr_idx][:]
            no_improve = 0
            history.append(best_distance)

    return best_distance, best_solution, history

In [ ]:
# Check data directory and load .npy files
DATA_DIR = './lab2'

import os
if not os.path.isdir(DATA_DIR):
    raise SystemExit(f"Data directory not found: {DATA_DIR}")

files = [f for f in os.listdir(DATA_DIR) if f.endswith('.npy')]
files.sort()
if not files:
    raise SystemExit("No .npy files found under ./lab2")

In [ ]:
DEFAULT_POP = 10 
DEFAULT_STEPS = 10000 
DEFAULT_MUT = 0.2

all_results = []

In [ ]:
for fname in files:
    path = os.path.join(DATA_DIR, fname)
    data = np.load(path, allow_pickle=False)
    n = data.shape[0]

    pop = DEFAULT_POP
    steps = DEFAULT_STEPS
    mut = DEFAULT_MUT

    ic(f"Running {fname} -> N={n}, pop={pop}, steps={steps}, mut={mut}")
    bestDistance, bestSolution, history = solve(
        data=data,
        populationSize=pop,
        maxSteps=steps,
        mutationProbability=mut,
    )

    print("\n== Result ==")
    print(f"file: {fname}")
    print(f"bestDistance: {bestDistance}")
    print(f"bestSolution: {bestSolution}")
    all_results.append((fname, bestDistance, bestSolution))

print("\n=== SUMMARY (FULL) ===")
for name, dist_val, sol in all_results:
    print(f"file: {name}")
    print(f"bestDistance: {dist_val}")
    print(f"bestSolution: {sol}")
    print("-")